In [ ]:
! pip install -q pyspark==3.1.2 spark-nlp
! pip install bnlp_toolkit
! pip install -U bnlp_toolkit
! pip install emoji
!pip install bangla-stemmer
!pip3 install -q pyspark==3.1.2 spark-nlp
!pip install tabulate
!pip install graphviz

     |████████████████████████████████| 212.4 MB 59 kB/s 
     |████████████████████████████████| 142 kB 11.7 MB/s 
     |████████████████████████████████| 198 kB 13.2 MB/s 
     |████████████████████████████████| 23.9 MB 1.5 MB/s 
     |████████████████████████████████| 1.2 MB 25.6 MB/s 
     |████████████████████████████████| 743 kB 51.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 175 kB 3.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=a1579dd916ed4923b91acf2d7752e3eb8f0327a802db43ac822e037d44a44840
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
from bnlp import NLTKTokenizer
from bnlp.corpus import stopwords, punctuations, letters, digits
from bnlp.corpus import stopwords
from bnlp.corpus.util import remove_stopwords
from bangla_stemmer.stemmer import stemmer
from gensim.models import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import emoji
import re
import nltk
import heapq
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from tabulate import tabulate
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


punkt not found. downloading...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
spark = sparknlp.start()

In [ ]:
dataset = pd.read_csv('drive/MyDrive/DataBank/AIProject/Dataset.csv')
dataset.head()

,text,Representational_Racism,Ideological_Racism,Discursive_Racism
0,এই গরীব ভিখারিদের হাতের খাবার খাওয়ার থেকে না ...,1,0,0
1,এত্ত গুলো ছেলের মধ্যে দুইটা মেয়ে কেমনে কি,1,0,0
2,নিজে নামাজ পড়েনা কিন্তু এমন জ্ঞান বিতরণ করে এক...,0,0,0
3,আপনার থেকে আপনার দাঁত গুলা সুন্দর লাগে আপু,1,0,0
4,শুধু টাকা আর ক্ষমতা বা উচ্চশিক্ষা দিয়ে কখনো সু...,0,0,0


In [ ]:
dataset.head()

,text,Representational_Racism,Ideological_Racism,Discursive_Racism
0,এই গরীব ভিখারিদের হাতের খাবার খাওয়ার থেকে না ...,1,0,0
1,এত্ত গুলো ছেলের মধ্যে দুইটা মেয়ে কেমনে কি,1,0,0
2,নিজে নামাজ পড়েনা কিন্তু এমন জ্ঞান বিতরণ করে এক...,0,0,0
3,আপনার থেকে আপনার দাঁত গুলা সুন্দর লাগে আপু,1,0,0
4,শুধু টাকা আর ক্ষমতা বা উচ্চশিক্ষা দিয়ে কখনো সু...,0,0,0


In [ ]:
dataset.shape

(586, 4)

In [ ]:
# remove numbers
def remove_numbers(text):
    text = "".join(i for i in text if i in ["।"]
            or 2432 <= ord(i) <= 2531 or ord(i) == 32)
    text = " ".join(text.split())
    return text

In [ ]:
# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [ ]:
def bangla_stemmer(text): 
    temp =[]
    for x in text: 
        stmr = stemmer.BanglaStemmer()
        stm = stmr.stem(x)
        temp.append(stm)
    return temp

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained("lemma", "bn") \
    .setInputCols(["token"]) \
    .setOutputCol("lemma")


nlp_pipeline = Pipeline(stages=[document_assembler, tokenizer, lemmatizer])

light_pipeline = LightPipeline(nlp_pipeline.fit(
    spark.createDataFrame([[""]]).toDF("text")))

data_lst = []
def lemmatize_bangla(text):
  results = light_pipeline.fullAnnotate(text)
  for k, v in results[0].items():
    for item in range(len(v)):
      data_lst.append(v[item].result)
  return data_lst

lemma download started this may take some time.
Approximate size to download 90.6 KB
[OK!]


In [ ]:
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)


In [ ]:
def join_text(text_list):
    temp =  ' '.join(text_list);
    return temp


In [ ]:
dataset = dataset.dropna(axis='rows')

In [ ]:
text = dataset['text'].tolist()

In [ ]:
print(text)

['এই গরীব ভিখারিদের হাতের খাবার খাওয়ার থেকে না খেয়ে থাকা ভালো', 'এত্ত গুলো ছেলের মধ্যে দুইটা মেয়ে কেমনে কি', 'নিজে নামাজ পড়েনা কিন্তু এমন জ্ঞান বিতরণ করে একদম ফরজগার।', 'আপনার থেকে আপনার দাঁত গুলা সুন্দর লাগে আপু', 'শুধু টাকা আর ক্ষমতা বা উচ্চশিক্ষা দিয়ে কখনো সুখী হওয়া যায় না।', 'কালো হয়ে হলুদ রংয়ের জামা কে পরে আজব! তোকে এসব রং মানায় না 🙄', 'এমন গয়না তো কাজের মেয়েরা পরে', 'মেয়েটা দেখতে একদম পরীর মত সুন্দর', 'রাতবিরেতে মেয়েদের নিয়ে আড্ডাবাজি করা সিলেটের সংস্কৃতির সাথে যায়না।', 'মনে হচ্ছে পানিতে আটার বস্তা ঝাঁপিয়ে পড়ল', 'আজকের পরীই আগামী দিনের শ্বাশুরী', 'মোটা মেয়ের অগ্রধিকার', 'আফা আপনার প্রোমোশন হয় না??', 'একটি নারীর কারণে জীবনটাই বরবাদ।', 'টাক মাথার সাথে নাকফুলটা কী সুন্দর মানিয়েছ!', 'এই গরীব ভিখারিদের হাতের খাবার খাওয়ার থেকে না খেয়ে থাকা ভালো', 'মেয়েদের চাইতে ছলনাময়ী বাংলাদেশ শিক্ষা মন্ত্রনালয়', 'এমন গয়না তো কাজের মেয়েরা পরে', '২৫০ গ্রাম হাওয়া দিলে উড়ে যাবি 😂', 'জলহস্তির মতো এক্কেবারে 🤣', 'বউ হবে নির্ভরশীল, থাকবে গোলামের মতো', 'পুরাই সুপুরুষ এর মত লাগতেছে', 'একটি নারীর কারণে

In [ ]:
#emoji remove
temp = []  
for x in text:
    temp.append(remove_emoji(x))

text = temp.copy()
temp.clear()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  


In [ ]:

#punctuation remove
for x in text:
    temp.append(remove_stopwords(x, punctuations))

text = temp.copy()
temp.clear()

for x in text:
    temp.append(join_text(x))

text = temp.copy()
temp.clear()

In [ ]:
# remove numbers
for x in text:
    temp.append(remove_numbers(x))


text = temp.copy()
temp.clear()

In [ ]:
#remove stopwords
temp = []
for x in text:
    temp.append(remove_stopwords(x, stopwords))

text = temp.copy()
temp.clear()

In [ ]:
#stemmer 
for x in text:
    temp.append(bangla_stemmer(x))

text = temp.copy()
temp.clear()

applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied third rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied first rules..
applied second rules..
applied second rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
a

In [ ]:
# rejoin the words as sentance
for x in text:
    temp.append(join_text(x))

text = temp.copy()
temp.clear()


print(text)

['গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো', 'এত্ত গুলো ছেলের দুই মায় কেমনে', 'নামাজ পড়েনা জ্ঞান বিতরণ একদম ফরজগ', 'দাঁত গুলা সুন্দর লাগে আপু', 'টাকা ক্ষমতা উচ্চশিক্ষা দিয়ে কখনো সুখী হওয়া যায়', 'কালো হয় হলুদ রংয় জামা আজব তোকে এসব রং মানায়', 'গয়না কাজ মায়', 'মেয়ে একদম পরীর মত সুন্দর', 'রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না', 'পানি আটা বস্তা ঝাঁপিয়ে পড়ল', 'আজক পরী দিন শ্বাশুরী', 'মোটা মেয়ের অগ্রধিক', 'আফা প্রোমোশন', 'নারীর কারণে জীবন বরবাদ', 'টাক মাথা সাথে নাকফুল সুন্দর মান', 'গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো', 'মেয়ে চাইতে ছলনাময়ী বাংলাদেশ শিক্ষা মন্ত্রনালয়', 'গয়না কাজ মায়', 'গ্রাম হাওয়া দিলে উড়ে যাবি', 'জলহস্তির এক্কেবারে', 'বউ নির্ভরশীল গোলামের', 'পুরা সুপুরুষ মত লাগত', 'নারীর কারণে জীবন বরবাদ', 'এরকম কুৎসিত চেহা দেমাক দেখা', 'নিম্নবিত্ত সংসারে স্বপ্ন', 'পানি আটা বস্তা ঝাঁপিয়ে পড়ল', 'দাঁত পরিষ্ক হাসি', 'কালো ছাল বউ পায়', 'মেয়েটা চাকমা', 'রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না', 'মায় দেয়াল মাঝে সুন্দর বাইর জগ', 'মায় দেয়াল মাঝে সুন্দর বাইর জগ', 'ক্লিয় 

In [ ]:
temp = []
for x in text:
  temp.append([x])


text = temp.copy()
temp.clear()
print(text)

[['গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো'], ['এত্ত গুলো ছেলের দুই মায় কেমনে'], ['নামাজ পড়েনা জ্ঞান বিতরণ একদম ফরজগ'], ['দাঁত গুলা সুন্দর লাগে আপু'], ['টাকা ক্ষমতা উচ্চশিক্ষা দিয়ে কখনো সুখী হওয়া যায়'], ['কালো হয় হলুদ রংয় জামা আজব তোকে এসব রং মানায়'], ['গয়না কাজ মায়'], ['মেয়ে একদম পরীর মত সুন্দর'], ['রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না'], ['পানি আটা বস্তা ঝাঁপিয়ে পড়ল'], ['আজক পরী দিন শ্বাশুরী'], ['মোটা মেয়ের অগ্রধিক'], ['আফা প্রোমোশন'], ['নারীর কারণে জীবন বরবাদ'], ['টাক মাথা সাথে নাকফুল সুন্দর মান'], ['গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো'], ['মেয়ে চাইতে ছলনাময়ী বাংলাদেশ শিক্ষা মন্ত্রনালয়'], ['গয়না কাজ মায়'], ['গ্রাম হাওয়া দিলে উড়ে যাবি'], ['জলহস্তির এক্কেবারে'], ['বউ নির্ভরশীল গোলামের'], ['পুরা সুপুরুষ মত লাগত'], ['নারীর কারণে জীবন বরবাদ'], ['এরকম কুৎসিত চেহা দেমাক দেখা'], ['নিম্নবিত্ত সংসারে স্বপ্ন'], ['পানি আটা বস্তা ঝাঁপিয়ে পড়ল'], ['দাঁত পরিষ্ক হাসি'], ['কালো ছাল বউ পায়'], ['মেয়েটা চাকমা'], ['রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না'], ['মায় দেয়াল ম

In [ ]:
# lemmetization
temp = []
for x in text:
  lemmas = lemmatize_bangla(x)
  temp.append(join_text(lemmas))
temp.clear()
print(text)


[['গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো'], ['এত্ত গুলো ছেলের দুই মায় কেমনে'], ['নামাজ পড়েনা জ্ঞান বিতরণ একদম ফরজগ'], ['দাঁত গুলা সুন্দর লাগে আপু'], ['টাকা ক্ষমতা উচ্চশিক্ষা দিয়ে কখনো সুখী হওয়া যায়'], ['কালো হয় হলুদ রংয় জামা আজব তোকে এসব রং মানায়'], ['গয়না কাজ মায়'], ['মেয়ে একদম পরীর মত সুন্দর'], ['রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না'], ['পানি আটা বস্তা ঝাঁপিয়ে পড়ল'], ['আজক পরী দিন শ্বাশুরী'], ['মোটা মেয়ের অগ্রধিক'], ['আফা প্রোমোশন'], ['নারীর কারণে জীবন বরবাদ'], ['টাক মাথা সাথে নাকফুল সুন্দর মান'], ['গরীব ভিখারি হাত খাবা খাওয় খেয়ে ভালো'], ['মেয়ে চাইতে ছলনাময়ী বাংলাদেশ শিক্ষা মন্ত্রনালয়'], ['গয়না কাজ মায়'], ['গ্রাম হাওয়া দিলে উড়ে যাবি'], ['জলহস্তির এক্কেবারে'], ['বউ নির্ভরশীল গোলামের'], ['পুরা সুপুরুষ মত লাগত'], ['নারীর কারণে জীবন বরবাদ'], ['এরকম কুৎসিত চেহা দেমাক দেখা'], ['নিম্নবিত্ত সংসারে স্বপ্ন'], ['পানি আটা বস্তা ঝাঁপিয়ে পড়ল'], ['দাঁত পরিষ্ক হাসি'], ['কালো ছাল বউ পায়'], ['মেয়েটা চাকমা'], ['রাতবিরে মেয়ে আড্ডাবাজি সিলেটের সংস্কৃতির সাথে যায়না'], ['মায় দেয়াল ম

In [ ]:
# for x in text:
#   print(x[0])

rep_lbl = dataset['Representational_Racism'].tolist();
ideo_lbl = dataset['Ideological_Racism'].tolist();
disc_lbl = dataset['Discursive_Racism'].tolist();

labeledDataset = [];
labels = [];

for i in range(len(text)):
  labels.append([int(rep_lbl[i]),int(ideo_lbl[i]),int(disc_lbl[i])])
  labeledDataset.append(
      {
          "text": text[i][0],
          "rep_lbl": rep_lbl[i],
          "ideo_lbl": ideo_lbl[i],
          "disc_lbl": disc_lbl[i] 
      }
  )

In [ ]:
print(labels)

[[1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [1, 0, 0], [0, 0, 0], [1, 1, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [1, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 1, 0], [1, 0, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 1, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 0, 1], [1, 0, 0], [0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0]

In [ ]:
#creating histogram

bnltk = NLTKTokenizer()
word2count={}
senctence2count={}
for data in text:
  words = bnltk.word_tokenize(data[0])
  for word in words:
    if word not in word2count.keys():
      word2count[word]=1
    else:
      word2count[word]+=1


print(word2count)  
print(len(word2count))   


{'গরীব': 8, 'ভিখারি': 7, 'হাত': 20, 'খাবা': 17, 'খাওয়': 7, 'খেয়ে': 8, 'ভালো': 12, 'এত্ত': 8, 'গুলো': 10, 'ছেলের': 11, 'দুই': 9, 'মায়': 71, 'কেমনে': 9, 'নামাজ': 3, 'পড়েনা': 1, 'জ্ঞান': 1, 'বিতরণ': 1, 'একদম': 4, 'ফরজগ': 1, 'দাঁত': 9, 'গুলা': 10, 'সুন্দর': 26, 'লাগে': 26, 'আপু': 20, 'টাকা': 8, 'ক্ষমতা': 3, 'উচ্চশিক্ষা': 1, 'দিয়ে': 4, 'কখনো': 4, 'সুখী': 1, 'হওয়া': 1, 'যায়': 6, 'কালো': 12, 'হয়': 8, 'হলুদ': 1, 'রংয়': 1, 'জামা': 3, 'আজব': 1, 'তোকে': 3, 'এসব': 2, 'রং': 2, 'মানায়': 2, 'গয়না': 10, 'কাজ': 11, 'মেয়ে': 7, 'পরীর': 1, 'মত': 14, 'রাতবিরে': 6, 'মেয়ে': 32, 'আড্ডাবাজি': 6, 'সিলেটের': 7, 'সংস্কৃতির': 6, 'সাথে': 18, 'যায়না': 7, 'পানি': 17, 'আটা': 10, 'বস্তা': 9, 'ঝাঁপিয়ে': 9, 'পড়ল': 9, 'আজক': 2, 'পরী': 1, 'দিন': 1, 'শ্বাশুরী': 1, 'মোটা': 18, 'মেয়ের': 9, 'অগ্রধিক': 8, 'আফা': 3, 'প্রোমোশন': 1, 'নারীর': 7, 'কারণে': 7, 'জীবন': 8, 'বরবাদ': 7, 'টাক': 13, 'মাথা': 9, 'নাকফুল': 7, 'মান': 8, 'চাইতে': 6, 'ছলনাময়ী': 6, 'বাংলাদেশ': 6, 'শিক্ষা': 7, 'মন্ত্রনালয়': 6, 'গ্রাম': 1, 'হাওয়া': 1, 'দিলে': 

In [ ]:
#finding out most frequent words
freq_words = heapq.nlargest(800,word2count,key=word2count.get)
print(freq_words)
print(len(freq_words))

['মায়', 'এক', 'মেয়ে', 'মার্কা', 'সুন্দর', 'লাগে', 'ছাল', 'তোমাকে', 'হাত', 'আপু', 'এক্কেবারে', 'চেহা', 'মেয়েটা', 'ক্ষেত', 'সাথে', 'মোটা', 'খাবা', 'পানি', 'স্বপ্ন', 'হাসি', 'তোর', 'গার্মেন্টস', 'মানুষ', 'বউ', 'মত', 'পর', 'জায়গা', 'দেখ', 'আপনা', 'টাক', 'ভালো', 'কালো', 'চায়', 'এত', 'বেটা', 'বিশ্ব', 'ঘুরে', 'ছেলের', 'কাজ', 'এরকম', 'পায়', 'রান্না', 'বাবা', 'অন্ধ', 'মিটে', 'শুনে', 'গুলো', 'গুলা', 'গয়না', 'আটা', 'লাগত', 'দেমাক', 'বাসায়', 'একটু', 'হই', 'রে', 'বড়', 'দিছ', 'বিয়ে', 'এদি', 'দুই', 'কেমনে', 'দাঁত', 'বস্তা', 'ঝাঁপিয়ে', 'পড়ল', 'মেয়ের', 'মাথা', 'জলহস্তির', 'কুৎসিত', 'দেখা', 'দুইজন', 'শাহজালাল', 'রোহিঙ্গাগুল', 'দক্ষিণান্ঞলে', 'টিকা', 'দাম', 'পোস্ট', 'ছাত্র', 'কর্মীর', 'নিগ্র', 'আয়', 'ধাক্কা', 'মারলে', 'ছোটখাটো', 'ভূমিকম্প', 'গরীব', 'খেয়ে', 'এত্ত', 'টাকা', 'হয়', 'অগ্রধিক', 'জীবন', 'মান', 'পুরা', 'সুপুরুষ', 'শাড়ি', 'হাতির', 'মতন', 'শরীর', 'মশা', 'রক্ত', 'খাই', 'বিশ্ববিদ্যালয়', 'বানা', 'ভাই', 'য়', 'লিঙ্গ', 'কথা', 'টিয়া', 'পাখি', 'গরু', 'পা', 'বেঁধে', 'ফাল', 'ব্লগ', 'কর্মী', 'বিরক্তিকর'

In [ ]:
#IDF Matrix

word_idfs = {}
for word in freq_words:
  doc_count=0
  for data in text:
    if word in bnltk.word_tokenize(data[0]):
      doc_count+=1
  word_idfs[word] = np.log((len(text)/doc_count)+1)

print(word_idfs)
print(len(word_idfs))


{'মায়': 2.225004141443295, 'এক': 2.4724841297894433, 'মেয়ে': 2.9607525546579643, 'মার্কা': 3.460789478864401, 'সুন্দর': 3.158635744490844, 'লাগে': 3.158635744490844, 'ছাল': 3.319132428607851, 'তোমাকে': 3.3640063533360753, 'হাত': 3.4111477125153233, 'আপু': 3.4111477125153233, 'এক্কেবারে': 3.460789478864401, 'চেহা': 3.460789478864401, 'মেয়েটা': 3.460789478864401, 'ক্ষেত': 4.087655574071304, 'সাথে': 3.51320244003865, 'মোটা': 3.51320244003865, 'খাবা': 3.568703852670969, 'পানি': 3.568703852670969, 'স্বপ্ন': 3.568703852670969, 'হাসি': 3.568703852670969, 'তোর': 3.6276687230690396, 'গার্মেন্টস': 3.568703852670969, 'মানুষ': 3.6276687230690396, 'বউ': 3.6905447334329975, 'মত': 3.7578723256008875, 'পর': 3.7578723256008875, 'জায়গা': 3.7578723256008875, 'দেখ': 3.7578723256008875, 'আপনা': 3.7578723256008875, 'টাক': 3.8303122406539125, 'ভালো': 3.9086841041626315, 'কালো': 3.9086841041626315, 'চায়': 3.9940218405942316, 'এত': 3.9086841041626315, 'বেটা': 3.9086841041626315, 'বিশ্ব': 3.9086841041626315, 'ঘ

In [ ]:
#TF Matrix
tf_matrix={}
for word in freq_words:
  doc_tf = []
  for data in text:
    frequency = 0
    for w in bnltk.word_tokenize(data[0]):
      if w == word:
        frequency += 1
    if len(bnltk.word_tokenize(data[0]))!= 0:
           tf_word = frequency/len(bnltk.word_tokenize(data[0]))
           doc_tf.append(tf_word) 
    tf_matrix[word] = doc_tf

print(len(tf_matrix))

800


In [ ]:
#TF-IDF calculation
tfidf_matrix = []
for word in tf_matrix.keys():
  tfidf = []
  for value in tf_matrix[word]:
    score = value*word_idfs[word]
    tfidf.append(score)
  tfidf_matrix.append(tfidf)

TFM = np.asarray(tfidf_matrix)
TFM = np.transpose(TFM)
print(TFM)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.37083402 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.74166805 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
#split
X_train,X_test,y_train,y_test = train_test_split(TFM,rep_lbl,test_size= 0.3,random_state = 2)

In [ ]:
#init evaluation metrics
evalution_table = []
evalution_table.append(['Classifier Name','Precision','Recall','Accuracy','F1-Score'])


In [ ]:
#Logistic Regression

classifier = LogisticRegression()
classifier.fit(X_train,y_train)
y_pred_LR = classifier.predict(X_test)
cm_LR = confusion_matrix(y_test,y_pred_LR)
pr_LR= precision_score(y_test, y_pred_LR, average=None)
re_LR= recall_score(y_test, y_pred_LR, average=None)
f1_LR= f1_score(y_test, y_pred_LR, average=None)
acc_LR = accuracy_score(y_test,y_pred_LR)
evalution_table.append(['Logistic Regression',pr_LR[0],re_LR[0],acc_LR,f1_LR[0]])
print(cm_LR)
print(acc_LR)

[[ 59  15]
 [  2 100]]
0.9034090909090909


In [ ]:
#decision tree
classifierDecisionTree = tree.DecisionTreeClassifier()
classifierDecisionTree.fit(X_train,y_train)
y_pred_DT = classifierDecisionTree.predict(X_test)
cm_DT = confusion_matrix(y_test,y_pred_DT)
pr_DT= precision_score(y_test, y_pred_DT, average=None)
re_DT= recall_score(y_test, y_pred_DT, average=None)
f1_DT= f1_score(y_test, y_pred_DT, average=None)
acc_DT = accuracy_score(y_test,y_pred_DT)
evalution_table.append(['Decision Tree',pr_DT[0],re_DT[0],acc_DT,f1_DT[0]])
print(cm_DT)
print(acc_DT)

[[69  5]
 [ 4 98]]
0.9488636363636364


In [ ]:
#SVM
classifierSVM = make_pipeline(StandardScaler(), SVC(gamma='auto'))
classifierSVM.fit(X_train,y_train)
y_pred_SVM = classifierSVM.predict(X_test)
cm_SVM = confusion_matrix(y_test,y_pred_SVM)
pr_SVM= precision_score(y_test, y_pred_SVM, average=None)
re_SVM= recall_score(y_test, y_pred_SVM, average=None)
f1_SVM= f1_score(y_test, y_pred_SVM, average=None)
acc_SVM = accuracy_score(y_test,y_pred_SVM)
evalution_table.append(['SVM',pr_SVM[0],re_SVM[0],acc_SVM,f1_SVM[0]])
print(cm_SVM)
print(acc_SVM)

[[32 42]
 [ 8 94]]
0.7159090909090909


In [ ]:
#Random Forest
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred_RF = rfc.predict(X_test)
cm_RF = confusion_matrix(y_test,y_pred_RF)
pr_RF= precision_score(y_test, y_pred_RF, average=None)
re_RF= recall_score(y_test, y_pred_RF, average=None)
f1_RF= f1_score(y_test, y_pred_RF, average=None)
acc_RF = accuracy_score(y_test,y_pred_RF)
evalution_table.append(['Random Forest',pr_RF[0],re_RF[0],acc_RF,f1_RF[0]])
print(cm_RF)
print(acc_RF)

[[71  3]
 [ 4 98]]
0.9602272727272727


In [ ]:
#KNN
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)
y_pred_KNN= neigh.predict(X_test)
cm_KNN = confusion_matrix(y_test,y_pred_KNN)
pr_KNN= precision_score(y_test, y_pred_KNN, average=None)
re_KNN= recall_score(y_test, y_pred_KNN, average=None)
f1_KNN= f1_score(y_test, y_pred_KNN, average=None)
acc_KNN = accuracy_score(y_test,y_pred_KNN)
evalution_table.append(['KNN',pr_KNN[0],re_KNN[0],acc_KNN,f1_KNN[0]])
print(cm_KNN)
print(acc_KNN)

[[73  1]
 [10 92]]
0.9375


In [ ]:
print(tabulate(evalution_table))

-------------------  ------------------  -------------------  ------------------  ------------------
Classifier Name      Precision           Recall               Accuracy            F1-Score
Logistic Regression  0.9672131147540983  0.7972972972972973   0.9034090909090909  0.874074074074074
Decision Tree        0.9452054794520548  0.9324324324324325   0.9488636363636364  0.9387755102040816
SVM                  0.8                 0.43243243243243246  0.7159090909090909  0.5614035087719298
Random Forest        0.9466666666666667  0.9594594594594594   0.9602272727272727  0.9530201342281879
KNN                  0.8795180722891566  0.9864864864864865   0.9375              0.9299363057324841
-------------------  ------------------  -------------------  ------------------  ------------------
